In [ ]:
%load_ext autoreload
%autoreload 2

import time
import asyncio
from ipywidgets import (
    HBox,
    VBox,
    Button,
    IntSlider,
    Dropdown,
    Checkbox,
    Label
)
from ipyevents import Event

from park.simulation import Simulation
from park.render import Camera, Renderer

In [ ]:
world_width = 10000
world_height = 10000

cam_width = 1100
cam_height = 700

sim = Simulation(
    world_width=world_width,
    world_height=world_height
)
cam = Camera(
    width=cam_width,
    height=cam_height,
    world_width=world_width,
    world_height=world_height
)
ren = Renderer(cam, tile_pixels=48)

In [ ]:
# start_btn = Button(description="Start", button_style="success")
# stop_btn = Button(description="Stop",  button_style="warning")
# speed = IntSlider(value=1, min=1, max=8, description="Speed")
info = Label("Mouse drag to pan, mouse wheel to zoom")

ui = VBox([
    ren.widget,
    # HBox([
        # start_btn,
        # stop_btn,
        # speed,
    # ]),
    info,
])

In [ ]:
# --- pan/zoom state ---
_dragging = False
_last_xy = (0.0, 0.0)
_acc_dx = 0.0
_acc_dy = 0.0

# --- frame timing ---
_target_fps = 60.0
_min_dt = 1.0 / _target_fps
_last_frame_t = 0.0

# Single render task reference
_render_task = None

async def _render_loop():
    """Consumes accumulated deltas at most ~target_fps, runs while dragging is True."""
    global _acc_dx, _acc_dy, _last_frame_t, _dragging
    _last_frame_t = 0.0
    while _dragging:
        now = time.monotonic()
        if now - _last_frame_t >= _min_dt:
            dx, dy = _acc_dx, _acc_dy
            _acc_dx = _acc_dy = 0.0
            if dx or dy:
                # screen px -> world units; drag moves the view
                cam.pan(-(dx) / cam.zoom, -(dy) / cam.zoom)
                # blit-only during pan
                ren.update_draw()
            _last_frame_t = now
        # yield to notebook event loop
        await asyncio.sleep(_min_dt)

# Attach pointer + wheel events to the MultiCanvas widget
_ev = Event(
    source=ren.mcanvas,
    watched_events=[
        "mousedown",
        "mouseup",
        "mousemove",
        "wheel",
    ],
    prevent_default_action=True,
    stop_propagation=True,
)

def _on_dom_event(e: dict):
    global _dragging, _last_xy, _acc_dx, _acc_dy, _render_task

    t = e.get("type", "")
    x = float(e.get("offsetX", 0.0))
    y = float(e.get("offsetY", 0.0))

    if t == "mousedown":
        _dragging = (e.get("buttons", 0) & 1) != 0  # left button
        _last_xy = (x, y)
        # start a loop if not running
        if (_render_task is None) or _render_task.done():
            _render_task = asyncio.create_task(_render_loop())
        return

    if t == "mouseup":
        _dragging = False
        return

    if t == "mousemove" and _dragging:
        lx, ly = _last_xy
        _acc_dx += (x - lx)
        _acc_dy += (y - ly)
        _last_xy = (x, y)
        return

    if t == "wheel":
        dy = float(e.get("deltaY", 0.0))
        ren.zoom_at(x, y, dy)
        ren.update_draw()

_ev.on_dom_event(_on_dom_event)

In [ ]:
ui